In [28]:
# Generalized + Individualized Approach
# Train & Test on Distinct Groups of Participants
# Also Use Small % of Test Participant's Data (Chronological Split)

# Evaluate "generalizability" of each train participant for particular test participant

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [30]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = False

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = False

# Number of Estimators for Bagging
num_estimators = 10

# Sample Percentage for Bagging
sample_percentage = 0.75

# Threshold to Use
threshold_to_use = 0.80
use_all_if_lower_than_threshold = True

# Percentage of Test Participant Data to Use
test_percentage_to_use = 0.3

# File Name for Model Results
output_file = 'multi4_af_xgb_results_use_all_if_less.csv'

metric_keys = ['accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1']

# 1 -- Load Data

In [31]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [32]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [33]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [34]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [35]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [36]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [37]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [38]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [39]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [40]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [41]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [42]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [43]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [44]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [45]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    val_data = []
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            val_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    val_data = pd.concat(val_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training & Validation Data 
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)

    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])        
    X_val = val_data.drop(columns=['engagement', 'session_num', 'participant'])  
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    y_train = train_data['engagement']
    y_val = val_data['engagement']
    y_test = test_data['engagement']

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_val[c] = (X_val[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_val[c] = (X_val[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_val[c] = X_val[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_val[c] = X_val[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_val, y_val, X_test, y_test

In [46]:
# MODEL HERE
# Multiple Participants Approach 4: 
# Approach 4: Use Small % of Test Participant's Data (Chronological Split)
# Iterate over each other participant & select most "similar" to test participant

# Inputs: X_train, y_train, X_val, y_val, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import neighbors

def model(X_train, y_train, X_test, y_test, is_verbose, use_bagging):    
    # Model Here: must create pred & scores arrays
    if use_bagging:
        clf = BaggingClassifier(XGBClassifier(n_estimators=100), n_estimators=num_estimators, max_samples=sample_percentage)
        clf.fit(X_train.values, y_train.values)
        scores = clf.predict_proba(X_test.values)[:,1]
        pred = [round(value) for value in scores]   
    else:
        clf = XGBClassifier(n_estimators=100)
        clf.fit(X_train.values, y_train.values)
        scores = clf.predict(X_test.values)
        pred = [round(value) for value in scores] 

    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if is_verbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc

        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [47]:
all_pair_results = {}
for test_part in part:
    all_pair_results[test_part] = {}
    for train_part in part:
        if (train_part == test_part):
            continue
            
        new_results = {}
        print('Train:', train_part, '\tTest:', test_part)
        
        # Train - Test Split (and preprocessing)
        X_train, y_train, X_val, y_val, X_test, y_test = split([train_part], [test_part], test_percentage_to_use)
        
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        X_val = X_val.reset_index(drop=True)
        y_val = y_val.reset_index(drop=True)
        
        # Evaluate on Val
        results = model(X_train, y_train, X_val, y_val, False, False)
        all_pair_results[test_part][train_part] = results['accuracy']

Train: 7 	Test: 5
Train: 9 	Test: 5
Train: 11 	Test: 5
Train: 12 	Test: 5
Train: 17 	Test: 5
Train: 18 	Test: 5
Train: 5 	Test: 7
Train: 9 	Test: 7
Train: 11 	Test: 7
Train: 12 	Test: 7
Train: 17 	Test: 7
Train: 18 	Test: 7
Train: 5 	Test: 9
Train: 7 	Test: 9
Train: 11 	Test: 9
Train: 12 	Test: 9
Train: 17 	Test: 9
Train: 18 	Test: 9
Train: 5 	Test: 11
Train: 7 	Test: 11
Train: 9 	Test: 11
Train: 12 	Test: 11
Train: 17 	Test: 11
Train: 18 	Test: 11
Train: 5 	Test: 12
Train: 7 	Test: 12
Train: 9 	Test: 12
Train: 11 	Test: 12
Train: 17 	Test: 12
Train: 18 	Test: 12
Train: 5 	Test: 17
Train: 7 	Test: 17
Train: 9 	Test: 17
Train: 11 	Test: 17
Train: 12 	Test: 17
Train: 18 	Test: 17
Train: 5 	Test: 18
Train: 7 	Test: 18
Train: 9 	Test: 18
Train: 11 	Test: 18
Train: 12 	Test: 18
Train: 17 	Test: 18


In [48]:
all_pair_results

{5: {7: 0.8193968850104938,
  9: 0.7620678228211643,
  11: 0.8178504363194521,
  12: 0.8692146249861924,
  17: 0.8021650281674583,
  18: 0.8179608969402408},
 7: {5: 0.7351646791595684,
  9: 0.7948608745031233,
  11: 0.7593696763202725,
  12: 0.7568143100511073,
  17: 0.7237365133446905,
  18: 0.8112578080636003},
 9: {5: 0.7075495678698526,
  7: 0.7711616675139806,
  11: 0.762709710218607,
  12: 0.7325876970005084,
  17: 0.7096466700559227,
  18: 0.757371631926792},
 11: {5: 0.7609289617486339,
  7: 0.7816691505216096,
  9: 0.7952682563338301,
  12: 0.8262543467461501,
  17: 0.745963735717834,
  18: 0.8134624937903626},
 12: {5: 0.7901483244826955,
  7: 0.7982054568760301,
  9: 0.8030580479765611,
  11: 0.8287859366416407,
  17: 0.7838308002197399,
  18: 0.8109320637245926},
 17: {5: 0.7523542600896861,
  7: 0.7929372197309417,
  9: 0.8160313901345292,
  11: 0.8173766816143497,
  12: 0.7724215246636771,
  18: 0.7556053811659192},
 18: {5: 0.7405072904009721,
  7: 0.8168286755771568,
 

In [49]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        for m in metric_keys:
            new_results[m] = []
        
        # Determine Train - Test Participants
        train_part_candidate = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part_candidate, '\tTest:', test_part)
        
        # Evaluate each test participant separately
        for test in test_part:
            # Lookup best train participants to use for this test participant
            train_part = []
            best_part = -1
            best_part_acc = -1
            for candidate in train_part_candidate:
                lookup_acc = all_pair_results[test][candidate]
                if (lookup_acc>threshold_to_use):
                    train_part.append(candidate)
                if (lookup_acc>best_part_acc):
                    best_part = candidate
                    best_part_acc = lookup_acc
            if len(train_part)==0:
                if use_all_if_lower_than_threshold:
                    train_part = train_part_candidate
                else:
                    train_part.append(best_part)
            print('For ', test, 'using train participants: ', train_part)
            
            # Train - Test Split (and preprocessing)
            X_train, y_train, X_val, y_val, X_test, y_test = split(train_part, [test], test_percentage_to_use)
            X_train = X_train.append(X_val)
            y_train = y_train.append(y_val)
            
            idx = np.random.permutation(len(X_train))
            X_train = X_train.iloc[idx]
            y_train = y_train.iloc[idx]
            X_train = X_train.reset_index(drop=True)
            y_train = y_train.reset_index(drop=True)
        
            # Evaluate for this test participant  
            results = model(X_train, y_train, X_test, y_test, True, True)
            print(results)
            for m in metric_keys:
                new_results[m].append(results[m])
        
        # Combine results across test participants
        final_results = {}
        for m in metric_keys:
            final_results[m] = np.mean(new_results[m])
        final_results['train_participants'] = ", ".join(str(x) for x in train_part_candidate)
        final_results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(final_results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
For  7 using train participants:  [5]
{'precision_1': 0.8758750673128702, 'f1_1': 0.9038621839399834, 'precision_0': 0.9006024096385542, 'recall_0': 0.8194988253719655, 'auc': 0.9420275212696233, 'accuracy': 0.8853925140775091, 'recall_1': 0.9336969001148105, 'f1_0': 0.8581385813858139}
For  9 using train participants:  [5]
{'precision_1': 0.6658653846153846, 'f1_1': 0.7630853994490359, 'precision_0': 0.922739244951712, 'recall_0': 0.7392731535756155, 'auc': 0.9114403150172069, 'accuracy': 0.7959970348406227, 'recall_1': 0.8935483870967742, 'f1_0': 0.8208799791720907}
For  11 using train participants:  [5]
{'precision_1': 0.7894300861497681, 'f1_1': 0.864477503628447, 'precision_0': 0.7427912341407151, 'recall_0': 0.3362924281984334, 'auc': 0.817785086295883, 'accuracy': 0.7835723598435462, 'recall_1': 0.9552927024859663, 'f1_0': 0.46297627606038816}
For  12 using train participants:  [5]
{'precision_1': 0.7561239193083573, 'f1_1': 0.815145631

{'precision_1': 0.7064327485380117, 'f1_1': 0.7984137475214805, 'precision_0': 0.8713264495631453, 'recall_0': 0.592972972972973, 'auc': 0.897073578137408, 'accuracy': 0.7607217573221757, 'recall_1': 0.9179331306990881, 'f1_0': 0.705693148922483}
For  18 using train participants:  [12]
{'precision_1': 0.7811051004636785, 'f1_1': 0.8653681506849314, 'precision_0': 0.7329059829059829, 'recall_0': 0.23238482384823847, 'auc': 0.8694297499596877, 'accuracy': 0.7771084337349398, 'recall_1': 0.9700095969289827, 'f1_0': 0.352880658436214}

Train: [17] 	Test: [5, 7, 9, 11, 12, 18]
For  5 using train participants:  [17]
{'precision_1': 0.8370646766169154, 'f1_1': 0.8720440557175251, 'precision_0': 0.5993975903614458, 'recall_0': 0.4316702819956616, 'auc': 0.8313694690233038, 'accuracy': 0.7963917525773195, 'recall_1': 0.9100743745774171, 'f1_0': 0.5018915510718789}
For  7 using train participants:  [17]
{'precision_1': 0.894378698224852, 'f1_1': 0.8808275058275058, 'precision_0': 0.8265613243039

{'precision_1': 0.7813717848791893, 'f1_1': 0.8623655913978494, 'precision_0': 0.69140625, 'recall_0': 0.23983739837398374, 'auc': 0.8321680151782324, 'accuracy': 0.7732104890148831, 'recall_1': 0.9620921305182342, 'f1_0': 0.3561368209255533}

Train: [5, 12] 	Test: [7, 9, 11, 17, 18]
For  7 using train participants:  [5, 12]
{'precision_1': 0.8881634970195856, 'f1_1': 0.8931068931068932, 'precision_0': 0.8588469184890656, 'recall_0': 0.845732184808144, 'auc': 0.948151500494036, 'accuracy': 0.8759523020867837, 'recall_1': 0.898105625717566, 'f1_0': 0.8522391004142829}
For  9 using train participants:  [5, 12]
{'precision_1': 0.7574024226110363, 'f1_1': 0.8257520176082171, 'precision_0': 0.9393055923668169, 'recall_0': 0.8309495896834701, 'auc': 0.9435263585826116, 'accuracy': 0.8591549295774648, 'recall_1': 0.9076612903225807, 'f1_0': 0.8818113958696193}
For  11 using train participants:  [12]
{'precision_1': 0.803347280334728, 'f1_1': 0.8756727173219009, 'precision_0': 0.79741379310344

{'precision_1': 0.8018394648829431, 'f1_1': 0.87436177972283, 'precision_0': 0.7908992416034669, 'recall_0': 0.381201044386423, 'auc': 0.8124403005856353, 'accuracy': 0.800376647834275, 'recall_1': 0.9613071371291099, 'f1_0': 0.514446793516561}
For  17 using train participants:  [7, 12]
{'precision_1': 0.6677407805227354, 'f1_1': 0.7824627648416194, 'precision_0': 0.8942774005819593, 'recall_0': 0.4983783783783784, 'auc': 0.8942164352802651, 'accuracy': 0.7288179916317992, 'recall_1': 0.9447821681864235, 'f1_0': 0.6400555362721277}
For  18 using train participants:  [7]
{'precision_1': 0.7522398976046809, 'f1_1': 0.8537926740686935, 'precision_0': 0.6914285714285714, 'recall_0': 0.08197831978319783, 'auc': 0.8538585538806444, 'accuracy': 0.7503543586109143, 'recall_1': 0.9870441458733206, 'f1_0': 0.1465778316172017}

Train: [7, 17] 	Test: [5, 9, 11, 12, 18]
For  5 using train participants:  [7, 17]
{'precision_1': 0.82917522747285, 'f1_1': 0.887666928515318, 'precision_0': 0.7188160676

{'precision_1': 0.790936891854323, 'f1_1': 0.868039664378337, 'precision_0': 0.6007067137809188, 'recall_0': 0.1843817787418655, 'auc': 0.8193373167952198, 'accuracy': 0.7770618556701031, 'recall_1': 0.9617985125084516, 'f1_0': 0.2821576763485477}
For  7 using train participants:  [18]
{'precision_1': 0.8660287081339713, 'f1_1': 0.8992547612475847, 'precision_0': 0.9007029876977153, 'recall_0': 0.8026624902114331, 'auc': 0.9486965584702235, 'accuracy': 0.8790990394170255, 'recall_1': 0.9351320321469575, 'f1_0': 0.8488612836438924}
For  11 using train participants:  [18]
{'precision_1': 0.7938538205980067, 'f1_1': 0.8682776162790699, 'precision_0': 0.7633069082672707, 'recall_0': 0.35195822454308096, 'auc': 0.8012913498924834, 'accuracy': 0.7899464001158917, 'recall_1': 0.9580994386527666, 'f1_0': 0.48177269478198714}
For  12 using train participants:  [9, 18]
{'precision_1': 0.8400793650793651, 'f1_1': 0.8651409889660809, 'precision_0': 0.8810735770476631, 'recall_0': 0.825314260944950

...

{'precision_1': 0.7960679773408864, 'f1_1': 0.8695177434030936, 'precision_0': 0.7669256381798002, 'recall_0': 0.3608355091383812, 'auc': 0.7983472082344887, 'accuracy': 0.792264232942199, 'recall_1': 0.9578989574979951, 'f1_0': 0.4907670454545454}
For  12 using train participants:  [9, 18]
{'precision_1': 0.8390302066772655, 'f1_1': 0.8633946830265848, 'precision_0': 0.8785219399538107, 'recall_0': 0.8244473342002601, 'auc': 0.9403166218048509, 'accuracy': 0.8572954496902371, 'recall_1': 0.8892165122156698, 'f1_0': 0.8506261180679786}

Train: [5, 11, 12, 17] 	Test: [7, 9, 18]
For  7 using train participants:  [5, 11, 12, 17]
{'precision_1': 0.9030704394942806, 'f1_1': 0.8815750808110492, 'precision_0': 0.8217967599410898, 'recall_0': 0.8739232576350823, 'auc': 0.9456505272564951, 'accuracy': 0.8665120900960583, 'recall_1': 0.8610792192881745, 'f1_0': 0.8470588235294118}
For  9 using train participants:  [5, 11, 12, 17]
{'precision_1': 0.7690120824449183, 'f1_1': 0.8175292784284095, 'p

{'precision_1': 0.7864140035330014, 'f1_1': 0.8732946946054392, 'precision_0': 0.8653846153846154, 'recall_0': 0.30548302872062666, 'auc': 0.8150563964480811, 'accuracy': 0.7941474721135738, 'recall_1': 0.9817562149157979, 'f1_0': 0.4515631030490158}
For  17 using train participants:  [9]
{'precision_1': 0.6938775510204082, 'f1_1': 0.7819548872180451, 'precision_0': 0.8385579937304075, 'recall_0': 0.5783783783783784, 'auc': 0.8828144253676168, 'accuracy': 0.7421548117154811, 'recall_1': 0.8956433637284701, 'f1_0': 0.6845809341010877}

Train: [7, 9, 17, 18] 	Test: [5, 11, 12]
For  5 using train participants:  [7, 17, 18]
{'precision_1': 0.825452422650321, 'f1_1': 0.8859649122807017, 'precision_0': 0.7136563876651982, 'recall_0': 0.351409978308026, 'auc': 0.8436421542960815, 'accuracy': 0.8123711340206186, 'recall_1': 0.9560513860716701, 'f1_0': 0.4709302325581395}
For  11 using train participants:  [18]
{'precision_1': 0.7950424222259191, 'f1_1': 0.8689880898263479, 'precision_0': 0.765

{'precision_1': 0.8648648648648649, 'f1_1': 0.899421328189584, 'precision_0': 0.9028268551236749, 'recall_0': 0.8003132341425215, 'auc': 0.9490441593610167, 'accuracy': 0.8790990394170255, 'recall_1': 0.9368541905855339, 'f1_0': 0.8484848484848485}
For  11 using train participants:  [12, 18]
{'precision_1': 0.7876447876447876, 'f1_1': 0.873973580863977, 'precision_0': 0.86608442503639, 'recall_0': 0.31070496083550914, 'auc': 0.8103440947569205, 'accuracy': 0.7954512530783717, 'recall_1': 0.9815557337610264, 'f1_0': 0.4573405073020753}

Train: [11, 12, 17, 18] 	Test: [5, 7, 9]
For  5 using train participants:  [11, 12, 17, 18]
{'precision_1': 0.8250438853130486, 'f1_1': 0.8845671267252196, 'precision_0': 0.7012987012987013, 'recall_0': 0.351409978308026, 'auc': 0.8487954647787755, 'accuracy': 0.8103092783505155, 'recall_1': 0.9533468559837728, 'f1_0': 0.4682080924855491}
For  7 using train participants:  [18]
{'precision_1': 0.8638646576790907, 'f1_1': 0.8994082840236686, 'precision_0':

{'precision_1': 0.8678894205209995, 'f1_1': 0.9011868617168093, 'precision_0': 0.9037785588752196, 'recall_0': 0.8054032889584964, 'auc': 0.949485262980921, 'accuracy': 0.881417687976151, 'recall_1': 0.9371412169919633, 'f1_0': 0.8517598343685299}
For  17 using train participants:  [9, 11]
{'precision_1': 0.6594556873691556, 'f1_1': 0.7809917355371901, 'precision_0': 0.9123173277661796, 'recall_0': 0.47243243243243244, 'auc': 0.8867076042607958, 'accuracy': 0.7228033472803347, 'recall_1': 0.9574468085106383, 'f1_0': 0.6225071225071225}

Train: [5, 9, 11, 17, 18] 	Test: [7, 12]
For  7 using train participants:  [18]
{'precision_1': 0.8644426793751655, 'f1_1': 0.8993251618234402, 'precision_0': 0.903140203449801, 'recall_0': 0.7995301487862176, 'auc': 0.9484730285084426, 'accuracy': 0.8789334216628023, 'recall_1': 0.9371412169919633, 'f1_0': 0.8481827622014538}
For  12 using train participants:  [9, 11, 18]
{'precision_1': 0.7544495715227423, 'f1_1': 0.8465236686390533, 'precision_0': 0.

{'precision_1': 0.829452852153667, 'f1_1': 0.8914607444479198, 'precision_0': 0.7567567567567568, 'recall_0': 0.3644251626898048, 'auc': 0.8560516060714061, 'accuracy': 0.8211340206185567, 'recall_1': 0.9634888438133874, 'f1_0': 0.4919472913616399}


In [50]:
end_results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
end_results = end_results.append(all_results, ignore_index=True, sort=True)

In [51]:
end_results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.798539,0.886895,0.652716,0.833798,0.841531,0.761577,0.565304,0.923885,"7, 9, 11, 12, 17, 18",5
1,0.803946,0.876341,0.576273,0.852118,0.838253,0.774936,0.481946,0.950517,"5, 9, 11, 12, 17, 18",7
2,0.796358,0.864417,0.538597,0.853731,0.759568,0.787001,0.462110,0.936920,"5, 7, 11, 12, 17, 18",9
3,0.822155,0.894370,0.670972,0.854767,0.818972,0.801319,0.603724,0.918914,"5, 7, 9, 12, 17, 18",11
4,0.814217,0.890763,0.611336,0.857801,0.828084,0.793867,0.521002,0.935769,"5, 7, 9, 11, 17, 18",12


In [52]:
end_results.to_csv(output_file, index=False)

## 